# Taller 6 - Hadoop - HDFS

Juan Navarro, <jsnavarroa@unal.edu.co>


# Instalación

```bash
cd "${HOME}/worskpace/BDA"

git clone https://github.com/jsnavarroa/docker-hadoop.git
cd docker-hadoop

# Install docker-compose
conda create --name py3 python=3.6
conda activate py3
conda install -c conda-forge docker-compose

# Run local
docker-compose -f docker-compose-local.yml build
docker-compose -f docker-compose-local.yml up -d

```

* Hadoop URLs:
  * NameNode http://localhost:9870/dfshealth.html#tab-overview.
  * HDFS hdfs://localhost:9800.  

In [12]:
%%bash

# Enviroment variables
#export JAVA_HOME=$(readlink -f /usr/bin/java | sed "s:bin/java::")

echo "JAVA_HOME=$JAVA_HOME"

mkdir -p "./data/taller6_out/"

JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre/


# Práctica HDFS

Después de instalado el cluster de Hadoop se ejecutan los siguientes comandos para la práctica:

In [13]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"

# Cleaning
$HADOOP_HOME/bin/hdfs dfs -rm -r /casa

# Create house
$HADOOP_HOME/bin/hdfs dfs -mkdir -p /casa/piso1/sala
$HADOOP_HOME/bin/hdfs dfs -mkdir -p /casa/piso1/cocina
$HADOOP_HOME/bin/hdfs dfs -mkdir -p /casa/piso2/alcoba

# Add furniture
$HADOOP_HOME/bin/hdfs dfs -put ./resources/mesa.jpg /casa/piso1/sala
$HADOOP_HOME/bin/hdfs dfs -put ./resources/estufa.jpg /casa/piso1/cocina
$HADOOP_HOME/bin/hdfs dfs -put ./resources/libro.doc /casa/piso2/alcoba
$HADOOP_HOME/bin/hdfs dfs -put ./resources/televisor.jpg /casa/piso2/alcoba

# Renovation
$HADOOP_HOME/bin/hdfs dfs -mkdir -p /casa/piso2/estudio
$HADOOP_HOME/bin/hdfs dfs -put ./resources/cama.jpg /casa/piso2/alcoba
$HADOOP_HOME/bin/hdfs dfs -mv /casa/piso2/alcoba/televisor.jpg /casa/piso1/sala/televisor.jpg
$HADOOP_HOME/bin/hdfs dfs -mv /casa/piso2/alcoba/libro.doc /casa/piso2/estudio/libro.doc

# Check
$HADOOP_HOME/bin/hdfs dfs -ls -R /casa

Deleted /casa
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:53 /casa/piso1
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:54 /casa/piso1/cocina
-rw-r--r--   3 juan supergroup       5194 2018-11-30 10:54 /casa/piso1/cocina/estufa.jpg
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:54 /casa/piso1/sala
-rw-r--r--   3 juan supergroup       3106 2018-11-30 10:54 /casa/piso1/sala/mesa.jpg
-rw-r--r--   3 juan supergroup       5688 2018-11-30 10:54 /casa/piso1/sala/televisor.jpg
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:54 /casa/piso2
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:54 /casa/piso2/alcoba
-rw-r--r--   3 juan supergroup       5859 2018-11-30 10:54 /casa/piso2/alcoba/cama.jpg
drwxr-xr-x   - juan supergroup          0 2018-11-30 10:54 /casa/piso2/estudio
-rw-r--r--   3 juan supergroup       9216 2018-11-30 10:54 /casa/piso2/estudio/libro.doc


# Ejercicio de conteo de términos

## I. Computar el valor de pi en paralelo en 5 nodos con 5 "samples"

In [14]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"
HADOOP_EXAMPLES_JAR="${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.1.1.jar"

$HADOOP_HOME/bin/hadoop --loglevel WARN jar $HADOOP_EXAMPLES_JAR pi 5 5


Number of Maps  = 5
Samples per Map = 5
Wrote input for Map #0
Wrote input for Map #1
Wrote input for Map #2
Wrote input for Map #3
Wrote input for Map #4
Starting Job
Job Finished in 1.562 seconds
Estimated value of Pi is 3.68000000000000000000


2018-11-30 10:54:16,884 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2018-11-30 10:54:16,926 WARN io.ReadaheadPool: Failed readahead on ifile
EBADF: Bad file descriptor
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.posix_fadvise(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.posixFadviseIfPossible(NativeIO.java:270)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX$CacheManipulator.posixFadviseIfPossible(NativeIO.java:147)
	at org.apache.hadoop.io.ReadaheadPool$ReadaheadRequestImpl.run(ReadaheadPool.java:208)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


## II. Frecuencia de Palabras

In [15]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"
HADOOP_EXAMPLES_JAR="${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.1.1.jar"

cd ./data

# Split file
split --additional-suffix=".txt" biblia.txt

mkdir -p ./archivos_biblia
mv xa*.txt ./archivos_biblia

# Upload files
$HADOOP_HOME/bin/hdfs dfs -mkdir -p biblia/input
$HADOOP_HOME/bin/hdfs dfs -put -f ./archivos_biblia/* biblia/input

$HADOOP_HOME/bin/hdfs dfs -ls biblia/input

# Execute wordcount
$HADOOP_HOME/bin/hdfs dfs -rm -r biblia/output
$HADOOP_HOME/bin/hadoop --loglevel WARN jar $HADOOP_EXAMPLES_JAR wordcount biblia/input biblia/output

# Show results
$HADOOP_HOME/bin/hdfs dfs -cat biblia/output/part-r-00000 | grep --contex=5 "^él"

Found 7 items
-rw-r--r--   3 juan supergroup     111448 2018-11-30 10:54 biblia/input/xaa.txt
-rw-r--r--   3 juan supergroup     127578 2018-11-30 10:54 biblia/input/xab.txt
-rw-r--r--   3 juan supergroup     151478 2018-11-30 10:54 biblia/input/xac.txt
-rw-r--r--   3 juan supergroup     190377 2018-11-30 10:54 biblia/input/xad.txt
-rw-r--r--   3 juan supergroup     167416 2018-11-30 10:54 biblia/input/xae.txt
-rw-r--r--   3 juan supergroup     131772 2018-11-30 10:54 biblia/input/xaf.txt
-rw-r--r--   3 juan supergroup     121707 2018-11-30 10:54 biblia/input/xag.txt
Deleted biblia/output
áspides	1
átate	1
échala	1
échalo	3
échate	4
él	592
él!	1
él,	215
él.	131
él:	8
él;	43
él?	8
éramos	5
ése	16
ése,	1
ésta	6
ésta,	4


2018-11-30 10:54:26,912 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2018-11-30 10:54:26,953 WARN io.ReadaheadPool: Failed readahead on ifile
EBADF: Bad file descriptor
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.posix_fadvise(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.posixFadviseIfPossible(NativeIO.java:270)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX$CacheManipulator.posixFadviseIfPossible(NativeIO.java:147)
	at org.apache.hadoop.io.ReadaheadPool$ReadaheadRequestImpl.run(ReadaheadPool.java:208)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
2018-11-30 10:54:26,955 WARN io.ReadaheadPool: Failed readahead on ifile
EBADF: Bad file descriptor
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.posix_fadvise(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSI

# Otros ejercicios

La implementación de los siguientes ejercicios se encuentra en (https://github.com/jsnavarroa/hadoop3-java-example). A continuación se muestran solo los resultados.

## 1. Calcular el tf*idf de cada término de la colección anterior (Biblia) usando mapreduce (se debe analizar y modificar  el archivo wordcount de java) . Donde tf es la frecuencia del término e idf es la frecuencia inversa del término en la colección de documentos. Se puede calcular como:

idf (t) =  log (|D| /(1+ numero de documentos donde aparece t)

donde D es el número de documentos en la colección.

## 2. Número de palabras/términos que comiencen con cada una de las vocales 

El mapper ignora los acentos y la capitalización. Por ejemplo, las palabras que comienzan con 'a', 'á', o 'A', se cuentan para 'a'.

In [16]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"

$HADOOP_HOME/bin/hdfs dfs -cat biblia/p2/part-r-00000

a	13108
e	19385
i	1331
o	3276
u	1687


## 3. Palabra más frecuente por cada una de las letras del abecedario

El mapper no distingue la diferencia entre mayúsculas y minúsculas.

In [17]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"

$HADOOP_HOME/bin/hdfs dfs -cat biblia/p3/part-r-00000 | grep --contex=5 "^e"

$HADOOP_HOME/bin/hdfs dfs -get -f biblia/p3/part-r-00000 ./data/taller6_out/otros_p3.txt

?	Dios?
a	cuales
b	recibirán
c	comienza
d	de
e	desconocida.
f	fue
g	ángel
h	muchos
i	higuera
j	Hijo


## 4. Los documentos donde la palabra más frecuente se encuentre.

## 5. Calcular la co-ocurrencia de todos los pares de términos en la colección de documentos. La salida del reducer debe ser: la clave (pareja de términos) y valor (la frecuencia en la que aparecen los dos términos).

## 6. Implementar un contador de kmer sobre un genoma usando hadoop. (Similar a word count, pero para identificar los kmers se realizan corrimientos sobre la cadena de ADN de acuerdo al tamaño) . La función debe recibir la cadena y el tamaño k.


In [18]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"

# Uncompress data
gzip -dkf ./data/ecoli.fa.gz

# Remove first line of file
tail -n +2 ./data/ecoli.fa > ./data/ecoli.txt

$HADOOP_HOME/bin/hdfs dfs -mkdir -p ecoli/input

# Upload data
$HADOOP_HOME/bin/hdfs dfs -put -f ./data/ecoli.txt ecoli/input

In [19]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"
K=3
OUTPUT_FILE="./data/taller6_out/ecoli-${K}-mers.txt"


$HADOOP_HOME/bin/hdfs dfs -cat ecoli/${K}-kmer/part-r-00000 > $OUTPUT_FILE

cat $OUTPUT_FILE | head

AAA	104317
AAC	79028
AAG	60678
AAT	79476
ACA	56161
ACC	71689
ACG	70094
ACT	47752
AGA	54161
AGC	77456


## 7. Usando el genoma de la ecoli que se encuentra en: ecoli.fa.gz

### 7.1  ¿Cuáles son los 10 más frecuentes 5-mers?

In [20]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"
K=5
OUTPUT_FILE="./data/taller6_out/ecoli-${K}-mers.txt"


$HADOOP_HOME/bin/hdfs dfs -cat ecoli/${K}-kmer-sort/part-r-00000 | sort -nbr > $OUTPUT_FILE

cat $OUTPUT_FILE | head

12410	CCAGC
12382	CGCCA
12033	GCCAG
11986	GCTGG
11971	CTGGC
11838	TGGCG
11326	CAGCG
11322	CAGCA
11111	CGCTG
10795	TTTTT


### 7.2  ¿Cuáles son los 10 más frecuentes 9-mers?

In [21]:
%%bash

HADOOP_HOME="${HOME}/Programas/BDA/hadoop-3.1.1"
K=9
OUTPUT_FILE="./data/taller6_out/ecoli-${K}-mers.txt"

$HADOOP_HOME/bin/hdfs dfs -cat ecoli/${K}-kmer-sort/part-r-00000 | sort -nbr > $OUTPUT_FILE

cat $OUTPUT_FILE | head

252	CCAGCGCCA
247	CAGCGCCAG
234	GCGCTGGCG
220	CGCCAGCAG
219	CGCTGGCGG
212	CTGGCGCTG
211	CGCCAGCGC
207	GCCAGCGCC
200	TGGCGCTGG
199	CCGCCAGCA
